# OLP_LLM: Pipeline for generating FOON graphs via LLM Prompting

## Step 1 :- Generate a FOON via LLM Prompting

1. Initialize libraries needed for FOON (``FOON_graph_analyser.py``) as well as OpenAI api.

2. Perform 2-stage prompting for recipe prototype.
    - In the first stage, we ask the LLM for a *high-level recipe* (list of instructions) and a *list of objects* needed for completing the recipe.
    - In the second stage, we ask the LLM for a breakdown of *state changes* that happen for each step of the recipe; specifically, we ask for the *preconditions* and *effects* of each action, which is similar to how a functional unit in FOON has *input* and *output object nodes*.

### Initialize library

In [1]:
from olp_library import *

### Language to  OLP

##### Select a Task

In [2]:
user_tasks = [
    "I am thirsty and in need of energy: how do I make coffee?",
    "I am thirsty: how do I make a lemonade?",
    "I am thirsty: how do I make a Bloody Mary cocktail?",

    # -- object stacking:
    "I have a red block, a green block, a yellow block, and a blue block on a table. How would I stack the blocks such that they are placed in alphabetical order?",
    "I have a red block, a green block, a yellow block, and a blue block on a table. How would I stack the blocks such that they are stacked in reverse alphabetical order?",
    "I have a red block, a green block, a yellow block, and a blue block on a table. How would I stack the blocks such that I have a pile of blocks of primary colours?",

    # -- object sorting:
    "I am packing Christmas gifts. I have 1 black toy, 2 pink toys, and 2 green toys. I also have a pink box and a green box. Which toys can I pack such that each box contains toys of the same colour?",
    "I have 3 yellow blocks, 2 red blocks, 1 green block, and 1 blue block. There is a red tile, a yellow tile, and a blue tile on the table. Which blocks can be stacked in their corresponding tiles based on colour?",

    # -- MineCraft domain:
    # "How can I make a bed in MineCraft?",
]

#### Perform Planning with LLM Prompting

In [3]:
openai_models = ["gpt-3.5-turbo", "gpt-4"]

# NOTE: all incontext examples will be stored within a JSON file:
# -- Q: can we randomly sample from the set of incontext examples?
# -- Q: should we also select an example "closest" to the provided task?
# incontext_file = "incontext_examples.txt"
incontext_file = "incontext_examples.json"

LLM_to_PDDL = True

user_task = user_tasks[-1]
selected_LLM = openai_models[-1]

llm_output = generate_OLP(user_task,
						  incontext_file,
						  selected_LLM, verbose=False)

print('Objects used in plan:', llm_output['RelevantObjects'])
print('Terminal Steps:', llm_output['TerminalSteps'])

json.dump(llm_output, open('raw_OLP.json', 'w'), indent=4)

Objects used in plan: ['first yellow block', 'second yellow block', 'third yellow block', 'yellow tile', 'first red block', 'second red block', 'red tile', 'blue block', 'blue tile']
Terminal Steps: [3, 5, 6]


In [4]:
## Visualize Outputs
print(f"***********************\n    High level plan\n***********************\n{llm_output['PlanSketch']}\n")
print("***********************\n   Object level plan\n***********************\n")
print(llm_output['OLP'], sep="\n")
print(f"\n***********************\n   Plan objects\n***********************\n{llm_output['RelevantObjects']}\n")

***********************
    High level plan
***********************
High-level Plan:
1. Pick and place the first yellow block on the yellow tile.
2. Pick and place the second yellow block on the first yellow block.
3. Pick and place the third yellow block on the second yellow block.
4. Pick and place the first red block on the red tile.
5. Pick and place the second red block on the first red block.
6. Pick and place the blue block on the blue tile.
unique_objects:["first yellow block", "second yellow block", "third yellow block", "yellow tile", "first red block", "second red block", "red tile", "blue block", "blue tile"]

***********************
   Object level plan
***********************

{'Query': 'Which blocks can be stacked in their corresponding tiles based on colour?', 'CompleteObjectSet': ['first yellow block', 'second yellow block', 'third yellow block', 'yellow tile', 'first red block', 'second red block', 'red tile', 'blue block', 'blue tile'], 'Instructions': [{'Step': 1, '

#### Sample FOON creation

In [5]:
# plan_step, plan_objects = llm_output['OLP'],

if '.json' in str(incontext_file).lower():
    sample_unit = create_functionalUnit_ver2(llm_output, index=0)
else:
    sample_unit = create_functionalUnit_ver1(llm_output['OLP'][0]['Instructions'],llm_output['RelevantObjects'])

sample_unit.print_functions[2](version=1)

Creating functional unit for Step 1: Pick and place the first yellow block on the yellow tile.
-> related objects: ['first yellow block', 'yellow tile']
Current object is : first yellow block
	 first yellow block state changes: {'Precondition': ['on table'], 'Effect': ['on yellow tile']}
		 Precondition : on table || related objects: table
		 Effect : on yellow tile || related objects: yellow tile
Current object is : yellow tile
	 yellow tile state changes: {'Precondition': ['on table'], 'Effect': ['under first yellow block']}
		 Precondition : on table || related objects: table
		 Effect : under first yellow block || related objects: first yellow block
O	first yellow block
S	on	 [table]
O	yellow tile
S	on	 [table]
M	Pick and Place	<Assumed>
O	first yellow block
S	on	 [yellow tile]
O	yellow tile
S	under	 [first yellow block]


#### Creating FOON units

In [6]:
FOON_prototype = []

for x in range(len(llm_output['OLP']['Instructions'])):

    new_unit = None

    if '.json' in str(incontext_file).lower():
        new_unit = create_functionalUnit_ver2(llm_output, index=x)

    # else:
    #     print(step['Step'])
    #     # -- now we will create functional units that follow the FOON format:
    #     new_unit = create_functionalUnit_ver1(step, plan_objects)

    # NOTE: in order to define a macro-problem, we need to properly identify all goal nodes;
    #       we will do this with the help of the LLM:
    if 'TerminalSteps' in llm_output and (x+1) in llm_output['TerminalSteps']:
        # -- set output objects as goal nodes for the functional units deemed as terminal steps:
        print(f'Functional unit #{x} has terminal goals!')
        for N in range(new_unit.getNumberOfOutputs()):
            new_unit.getOutputNodes()[N].setAsGoal()

    # elif x == (len(llm_output['OLP']['Instructions']) - 1):
    #     # -- by default, treat the last functional unit as the terminal unit:
    #     for N in range(new_unit.getNumberOfOutputs()):
    #         new_unit.getOutputNodes()[N].setAsGoal()

    # -- add the functional unit to the FOON prototype:

    if not new_unit.isEmpty():
        # -- we should only add a new functional unit if it is not empty, meaning it must have the following:
        #    1. >=1 input node and >= 1 output node
        #    2. a valid motion node
        FOON_prototype.append(new_unit)
        FOON_prototype[-1].print_functions[-1]()
    else:
        print('NOTE: the following functional unit has an error, so skipping it:')
        new_unit.print_functions[-1]()

    print()


Creating functional unit for Step 1: Pick and place the first yellow block on the yellow tile.
-> related objects: ['first yellow block', 'yellow tile']
Current object is : first yellow block
	 first yellow block state changes: {'Precondition': ['on table'], 'Effect': ['on yellow tile']}
		 Precondition : on table || related objects: table
		 Effect : on yellow tile || related objects: yellow tile
Current object is : yellow tile
	 yellow tile state changes: {'Precondition': ['on table'], 'Effect': ['under first yellow block']}
		 Precondition : on table || related objects: table
		 Effect : under first yellow block || related objects: first yellow block
O	first yellow block
S	on	 [table]
O	yellow tile
S	on	 [table]
M	Pick and Place	<Assumed>
O	first yellow block
S	on	 [yellow tile]
O	yellow tile
S	under	 [first yellow block]

Creating functional unit for Step 2: Pick and place the second yellow block on the first yellow block.
-> related objects: ['second yellow block', 'first yellow b

#### Testing PDDL Operator Generation via LLM prompting

In [7]:
if LLM_to_PDDL:

    pddl_sys_prompt_file = "llm_prompts/pddl_system_prompt.txt"

    pddl_system_prompt = open(pddl_sys_prompt_file, 'r').read()

    message = [{"role": "system", "content": pddl_system_prompt}]

    # for x in range(len(OLP_results['OLP']['Instructions'])):
    pddl_user_prompt = f"Generate PDDL for :\n{llm_output['OLP']['Instructions']}"

    message.extend([{"role": "user", "content": pddl_user_prompt}])

    _, response = prompt_LLM(given_prompt=message,
                             model_name=openai_models[-1], )
    print(response)


(:action pick_and_place_first_yellow_block
	:parameters ()
	:precondition (and (on first_yellow_block table) (on yellow_tile table))
	:effect (and (on first_yellow_block yellow_tile) (under yellow_tile first_yellow_block))
)

(:action pick_and_place_second_yellow_block
	:parameters ()
	:precondition (and (on second_yellow_block table) (on first_yellow_block yellow_tile))
	:effect (and (on second_yellow_block first_yellow_block) (under first_yellow_block second_yellow_block))
)

(:action pick_and_place_third_yellow_block
	:parameters ()
	:precondition (and (on third_yellow_block table) (under second_yellow_block third_yellow_block))
	:effect (and (on third_yellow_block second_yellow_block) (under second_yellow_block third_yellow_block))
)

(:action pick_and_place_first_red_block
	:parameters ()
	:precondition (and (on first_red_block table) (on red_tile table))
	:effect (and (on first_red_block red_tile) (under red_tile first_red_block))
)

(:action pick_and_place_second_red_block
	:par

### Writing FOON to a Text File

In [8]:
temp_file_name = 'prototype.txt'
task_file_name = f"{str.lower(re.compile('[^a-zA-Z]').sub('_', llm_output['OLP']['Query'])[:-1])}.txt"
print(task_file_name)

which_blocks_can_be_stacked_in_their_corresponding_tiles_based_on_colour.txt


In [9]:
# -- save the prototype FOON graph as a text file, which we will then run with a parser to correct numbering:
if not os.path.exists('./preprocess/'):
    os.makedirs('./preprocess/')

if not os.path.exists('./postprocess/'):
    os.makedirs('./postprocess/')

temp_file = open(f'./preprocess/{temp_file_name}', 'w')
temp_file.write('#\tFOON Prototype\n#\t-- Task Prompt: {0}\n//\n'.format(user_task))
for unit in FOON_prototype:
    unit.print_functions[2]()
    temp_file.write(unit.getFunctionalUnitText())
    print('//')

temp_file.close()

O	first yellow block
S	on	 [table]
O	yellow tile
S	on	 [table]
M	Pick and Place	<Assumed>
O	first yellow block
S	on	 [yellow tile]
O	yellow tile
S	under	 [first yellow block]
//
O	second yellow block
S	on	 [table]
O	first yellow block
S	on	 [yellow tile]
M	Pick and Place	<Assumed>
O	second yellow block
S	on	 [first yellow block]
O	first yellow block
S	under	 [second yellow block]
//
O	third yellow block
S	on	 [table]
O	second yellow block
S	under	 [third yellow block]
M	Pick and Place	<Assumed>
O	third yellow block
S	on	 [second yellow block]
//
O	first red block
S	on	 [table]
O	red tile
S	on	 [table]
M	Pick and Place	<Assumed>
O	first red block
S	on	 [red tile]
O	red tile
S	under	 [first red block]
//
O	second red block
S	on	 [table]
O	first red block
S	on	 [red tile]
M	Pick and Place	<Assumed>
O	second red block
S	on	 [first red block]
O	first red block
S	under	 [second red block]
//
O	blue block
S	on	 [table]
O	blue tile
S	on	 [table]
M	Pick and Place	<Assumed>
O	blue block
S	on	 [b

## Step 2 :- FOON to PDDL
1. Parse FOON file -- this step is important to ensure that all labels are unique and that the generated file follows the FOON syntax.

2. (Optional) Visualize FOON graph

3. Run ``FOON_to_PDDL.py`` script to generate FOON macro-operators

### Parse and clean generated FOON

In [10]:
# -- running parsing module to ensure that FOON labels and IDs are made consistent for further use:
#		(it is important that each object and state type have a *UNIQUE* identifier)
fpa.skip_JSON_conversion = True		# -- we don't need JSON versions of a FOON
fpa.skip_index_check = True			# -- always create a new set of index files

fpa.source_dir = './preprocess/'
fpa.target_dir = './postprocess/'
fpa._run_parser()

-- [FOON_parser] : Initiating parsing procedure!


 -- [FOON_parser] : Commencing parsing...
  -- parsing 'prototype.txt'...

 -- [FOON_parser] : Saving corrected files to './postprocess/'...
  -- Saving 'prototype.txt'...

-- Revising object label senses using WordNet and Concept-Net...


SUMMARY OF CHANGES:
  -- new total of OBJECTS : 10
  -- new total of STATES : 2
  -- new total of MOTIONS : 1

 -- [FOON_parser] : Parsing complete!


In [11]:
with open(f'./postprocess/{temp_file_name}', 'r') as input_file:
    with open(f'./postprocess/{task_file_name}', 'w') as output_file:
        for line in input_file:
            output_file.write(line)

### (Optional) Run FOON visualization tool

In [12]:
# -- after running the parser, take the 'prototype.txt' file and plot it using the following tool: https://davidpaulius.github.io/foon-view/
# fga._startFOONview()

### Generate PDDL files using ```FOON_to_PDDL``` module

In [13]:
import FOON_TAMP as fta
from pathlib import Path

FOON_subgraph_file = f'./postprocess/{task_file_name}'

# -- definition of macro and micro plan file names:
macro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_macro.plan'
micro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_micro.plan'

fta.micro_domain_file = './FOON_skills.pddl'

# -- create a new folder for the generated problem files and their corresponding plans:
fta.micro_problems_dir = './micro_problems-' + Path(FOON_subgraph_file).stem
if not os.path.exists(fta.micro_problems_dir):
    os.makedirs(fta.micro_problems_dir)

ftp = fta.ftp

# -- perform conversion of the FOON subgraph file to PDDL:
ftp.FOON_subgraph_file = FOON_subgraph_file
ftp._convert_to_PDDL('OCP')

## -- parse through the newly created domain file and find all (macro) planning operators:
all_macro_POs = {PO.name : PO for PO in fta._parseDomainPDDL(ftp.FOON_domain_file)}

# -- checking to see if a macro level plan has been found (FD allows exporting to a file):
outcome = fta._findPlan(
    domain_file=ftp.FOON_domain_file,   # NOTE: FOON_to_PDDL object will already contain the names of the
    problem_file=ftp.FOON_problem_file,  #   corresponding macro-level domain and problem files
    output_plan_file=macro_plan_file
)

# TODO: write planning pipeline for other planners?

complete_micro_plan = []

if fta._checkPlannerOutput(output=outcome):
    print("  -- [FOON-TAMP] : A macro-level plan for '" + str(FOON_subgraph_file) + "' was found!")

    # -- now that we have a plan, we are going to parse it for the steps:
    macro_file = open(macro_plan_file, 'r')

    # NOTE: counters for macro- and micro-level steps:
    macro_count, micro_count = 0, 0

    macro_plan_lines = list(macro_file)
    for L in macro_plan_lines:
        if L.startswith('('):
            print('\t\t\t' + str(macro_count) + ' : ' + L.strip())
    print()

    for L in range(len(macro_plan_lines)):

        if macro_plan_lines[L].startswith('('):
            # NOTE: this is where we have identified a macro plan's step; here, we check the contents of its PO definition for:
            #	1. preconditions - this will become a sub-problem file's initial states (as predicates)
            #	2. effects - this will become a sub-problem file's goal states (as predicates)

            macro_count += 1

            macro_PO_name = macro_plan_lines[L][1:-2].strip()

            print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

            # -- try to find this step's matching planning operator definition:
            matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

            # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
            if matching_PO_obj:
                # -- create sub-problem file (i.e., at the micro-level):
                micro_problem_file = fta._defineMicroProblem(
                    macro_PO_name,
                    preconditions=matching_PO_obj.getPreconditions(),
                    effects=matching_PO_obj.getEffects(),
                )

                micro_domain_file = fta._defineMicroDomain(
                    preconditions=matching_PO_obj.getPreconditions(),
                )

                complete_micro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')

                need_to_replan = False

                # -- try to find a sub-problem plan / solution:
                outcome = fta._findPlan(
                    domain_file=micro_domain_file,
                    problem_file=micro_problem_file,
                    output_plan_file=str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan')
                )

                print('\n\t' + 'step ' + str(macro_count) +' -- (' + macro_PO_name + ')')

                if fta._checkPlannerOutput(outcome):

                    print('\t\t -- micro-level plan found as follows:')

                    # -- open the micro problem file, read each line referring to a micro PO, and save to list:
                    micro_file = open(
                        str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan'), 'r')

                    # -- all except for the last line should be valid steps:
                    micro_file_lines = []
                    count = 0

                    for micro_line in micro_file:
                        if micro_line.startswith('('):
                            # -- parse the line and remove trailing newline character:
                            micro_step = micro_line.strip()
                            micro_file_lines.append(micro_step)

                            # -- print entire plan to the command line in format of X.Y,
                            #       where X is the macro-step count and Y is the micro-step count:
                            count += 1
                            print('\t\t\t' + str(macro_count) + '.' + str(count) + ' : ' + micro_step)
                        #endif
                    #endfor
        print()
else:
    print("  -- [FOON-TAMP] : A macro-level plan for '" + str(FOON_subgraph_file) + "' was NOT found!")


  -- Reading file line : 100%|██████████| 59/59 [00:00<00:00, 12931.18it/s]


< FOON_TAMP: converting FOON graph to PDDL planning problem (last updated: 31st March, 2023)>


 -- [FOON-fga] : Opening FOON file named './postprocess/which_blocks_can_be_stacked_in_their_corresponding_tiles_based_on_colour.txt'...
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unc

KeyboardInterrupt: 

In [14]:
complete_micro_plan = []

# -- we can also perform a search where we assume we already have a task tree
#		and we execute each functional unit one by one as we see

macro_plan_lines = []
for N in range(len(ftp.fga.FOON_lvl3)):
    PO_name = f'{ftp._reviseObjectLabels(ftp.fga.FOON_lvl3[N].getMotion().getMotionLabel())}_{N}'
    macro_plan_lines.append(PO_name)

macro_count = 0

for L in range(len(macro_plan_lines)):

    macro_count += 1

    macro_PO_name = macro_plan_lines[L]

    print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

    # -- try to find this step's matching planning operator definition:
    matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

    # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
    if matching_PO_obj:
        # -- create sub-problem file (i.e., at the micro-level):
        micro_problem_file = fta._defineMicroProblem(
            macro_PO_name,
            preconditions=matching_PO_obj.getPreconditions(),
            effects=matching_PO_obj.getEffects(),
        )

        micro_domain_file = fta._defineMicroDomain(
            preconditions=matching_PO_obj.getPreconditions(),
        )

        complete_micro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')

        need_to_replan = False

        # -- try to find a sub-problem plan / solution:
        outcome = fta._findPlan(
            domain_file=micro_domain_file,
            problem_file=micro_problem_file,
            output_plan_file=str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan')
        )

        print('\n\t' + 'step ' + str(macro_count) +' -- (' + macro_PO_name + ')')

        if fta._checkPlannerOutput(outcome):

            print('\t\t -- micro-level plan found as follows:')

            # -- open the micro problem file, read each line referring to a micro PO, and save to list:
            micro_file = open(
                str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan'), 'r')

            # -- all except for the last line should be valid steps:
            micro_file_lines = []
            count = 0

            for micro_line in micro_file:
                if micro_line.startswith('('):
                    # -- parse the line and remove trailing newline character:
                    micro_step = micro_line.strip()
                    micro_file_lines.append(micro_step)

                    # -- print entire plan to the command line in format of X.Y,
                    #       where X is the macro-step count and Y is the micro-step count:
                    count += 1
                    print('\t\t\t' + str(macro_count) + '.' + str(count) + ' : ' + micro_step)
                #endif
            #endfor
    print()


 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_0' macro-PO...
python3 C:/Users/david/fast-downward-23.06/fast-downward.py --plan-file ./micro_problems-which_blocks_can_be_stacked_in_their_corresponding_tiles_based_on_colour/pick_and_place_0_micro.plan ./micro_problems-which_blocks_can_be_stacked_in_their_corresponding_tiles_based_on_colour\./FOON_skills.pddl ./micro_problems-which_blocks_can_be_stacked_in_their_corresponding_tiles_based_on_colour/pick_and_place_0_problem.pddl --search astar(lmcut())
  -- [FOON-TAMP] : Error with planner execution! (planner used: fast-downward)
     -- Error code: 30 -- b"INFO     planner time limit: None\r\nINFO     planner memory limit: None\r\n\r\nINFO     Running translator.\r\nINFO     translator stdin: None\r\nINFO     translator time limit: None\r\nINFO     translator memory limit: None\r\nINFO     translator command line string: 'c:\\Python\\Python3\\python3.exe' 'C:\\Users\\david\\fast-downward-23.06\\builds\\release\\bin\

## Step 3: Connection with UTAMP System

In [129]:
import utamp.driver as driver
from utamp.generate import randomize_blocks

run_utamp = True

if run_utamp:
    # -- load the scene only once:
    # utamp = driver.UTAMP(scene_file_name='./utamp/scenes/panda_blocks_tiles-5.ttt')

    fpath = randomize_blocks('./utamp/scenes/panda_blocks_tiles_prototype.ttt')
    utamp = driver.UTAMP(scene_file_name=fpath)

    print("objects presently in scene:", utamp.objects_in_sim)

red_tile on Dummy
87 on 72
blue_tile on Dummy
88 on 74
yellow_tile on Dummy
89 on 83
green_tile on Dummy
90 on 76
red_block_1 on Dummy
91 on 71
red_block_2 on Dummy
92 on 73
red_block_3 on Dummy
93 on 67
red_block_4 on Dummy
94 on 82
red_block_5 on red_block_2
95 on 92
blue_block_1 on Dummy
96 on 79
blue_block_2 on Dummy
97 on 78
blue_block_3 on red_tile
98 on 87
blue_block_4 on Dummy
99 on 70
yellow_block_1 on Dummy
100 on 84
yellow_block_2 on yellow_block_1
101 on 100
yellow_block_3 on green_tile
102 on 90
green_block_1 on Dummy
103 on 81
green_block_2 on green_block_1
104 on 103
green_block_3 on green_block_2
105 on 104
green_block_4 on Dummy
106 on 80
green_block_5 on red_block_4
107 on 94
objects presently in scene: ['red_tile', 'blue_tile', 'yellow_tile', 'green_tile', 'red_block_1', 'red_block_2', 'red_block_3', 'red_block_4', 'red_block_5', 'blue_block_1', 'blue_block_2', 'blue_block_3', 'blue_block_4', 'yellow_block_1', 'yellow_block_2', 'yellow_block_3', 'green_block_1', 'gre

In [124]:
utamp.path_planning(target_object='red_block_1', gripper_action=1)

KeyboardInterrupt: 

In [125]:
if run_utamp:
    # -- now that we have a plan, we are going to parse it for the steps:
    macro_file = open(macro_plan_file, 'r')

    # NOTE: counters for macro- and micro-level steps:
    macro_count, micro_count = 0, 0

    # macro_plan_lines = list(macro_file)
    # for L in macro_plan_lines:
    #     if L.startswith('('):
    #         print('\t' + str(macro_count) + ' : ' + L.strip())
    #         macro_count += 1

    macro_plan_lines = []
    for N in range(len(ftp.fga.FOON_lvl3)):
        PO_name = f'{ftp._reviseObjectLabels(ftp.fga.FOON_lvl3[N].getMotion().getMotionLabel())}_{N}'
        # print('\t' + str(macro_count) + ' : ' + PO_name)
        macro_plan_lines.append(PO_name)

    macro_count = 0

    for L in range(len(macro_plan_lines)):
        # NOTE: this is where we have identified a macro plan's step; here, we check the contents of its PO definition for:
        #	1. preconditions - this will become a sub-problem file's initial states (as predicates)
        #	2. effects - this will become a sub-problem file's goal states (as predicates)

        macro_count += 1

        # macro_PO_name = macro_plan_lines[L][1:-2].strip()
        macro_PO_name = macro_plan_lines[L]

        print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

        # -- try to find this step's matching planning operator definition:
        matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

        # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
        if matching_PO_obj:
            # -- create sub-problem file (i.e., at the micro-level):
            micro_problem_file = fta._defineMicroProblem(
                macro_PO_name,
                preconditions=matching_PO_obj.getPreconditions(),
                effects=matching_PO_obj.getEffects(),
            )

            complete_micro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')
        else:
            continue

        goal_preds, objects_in_OLP = [], []

        # -- parse the goals in the generated micro-problem file:
        micro_problem_lines = open(micro_problem_file, 'r').readlines()
        for M in range(len(micro_problem_lines)):
            if ("(:goal" in micro_problem_lines[M]):
                while True:
                    M += 1

                    if micro_problem_lines[M].startswith("))"):
                        break

                    goal_string = str(micro_problem_lines[M]).replace("\t", "").strip()
                    if goal_string:
                        # -- first, we check if a goal string line is actually referring to a comment or a negation predicate:
                        if goal_string.startswith(";") or goal_string.startswith("(not"):
                            # -- we want to skip those:
                            continue

                        # -- append the goal string to the list of goal predicates:
                        goal_preds.append(goal_string)

                        # -- we also want to parse the goal string to identify all possible expressions for objects at the object level:
                        predicate_args = goal_preds[-1][1:-1].split(" ")
                        objects_in_OLP.extend([predicate_args[x] for x in range(1, len(predicate_args))])
                #endwhile
            #endif
        #endfor

        # print("objects in OLP:", objects_in_OLP)

        assert bool(goal_preds), f"Error: empty list of goals! Check the generated micro-problem file '{micro_problem_file}'"

        # -- prompt LLMs to perform object grounding
        #       (remove any objects that do not require grounding -- these are handled by the task planner system):
        objects_in_OLP = list(set(objects_in_OLP) - set(['hand', 'air', 'table', 'robot']))

        message = [
            {
                "role": "user",
                "content": f"Assign each of these simulation objects to real-world objects. You may only assign each object once."
                    " Give your output as a Python dictionary."
                    f"\nReal-world object: {objects_in_OLP}"
                    f"\nSimulated objects: {utamp.objects_in_sim}"
                    "\nExample: {'object_1': 'sim_object_1', 'object_2': 'sim_object_2'}"
            }
        ]

        _, response = prompt_LLM(message, selected_LLM)
        # print("LLM response:", response)
        regex_matches = re.findall(r'\{.+\}', str(eval(response)))

        if bool(regex_matches):
            # -- this means that the LLM has proposed some object groundings for us:
            obj_grounding = eval(regex_matches.pop())
            print(obj_grounding)

            # -- parse the goal predicates and replace the generic object names with those of the sim objects:
            print("before grounding:", goal_preds)
            for G in range(len(goal_preds)):
                goal_pred_parts = goal_preds[G][1:-1].split(" ")
                for obj in goal_pred_parts[1:]:
                    if obj in obj_grounding:
                        goal_preds[G] = goal_preds[G].replace(obj, obj_grounding[obj])

            print("after grounding:", goal_preds)

        driver.main(utamp, goal_preds=goal_preds)


 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_0' macro-PO...
{'yellow_tile': 'yellow_tile', 'first_yellow_block': 'yellow_block_1'}
before grounding: ['(in hand air)', '(on yellow_tile first_yellow_block)', '(under first_yellow_block yellow_tile)']
after grounding: ['(in hand air)', '(on yellow_tile yellow_block_1)', '(under yellow_block_1 yellow_tile)']
user 51
yellow_tile:on,yellow_block_1;yellow_block_1:under,yellow_tile:on,air;
robot:x,0.0:y,0.0:z,0.0:roll,-0.0:pitch,0.0:yaw,-0.0:dx,0.0:dy,0.0:dz,0.0;hand:x,0.5613500000000001:y,0.1121:z,0.11909000000000014:roll,3.1415853071795867:pitch,7.112306959198601e-16:yaw,0.3514800000000002:dx,0.024275404392629724:dy,0.2044156506692143:dz,0.05569436541288881;worksurface:x,0.5100384449958526:y,1.6720353035459823e-07:z,-0.0759137287140157:roll,0.0:pitch,0.0:yaw,-3.141590299206751:dx,0.6:dy,0.7:dz,0.0;red_block_1:x,0.6850385270012678:y,-0.25014950229540894:z,-0.05091371753810503:roll,6.2112586826695305e-19:pitch,-1.9013138

KeyboardInterrupt: 

# Baseline: LLM-as-Planner 

In [110]:
import utamp.driver as utamp

run_utamp = True

if run_utamp:
    # -- load the scene only once:
    # sim, _ = utamp.loadScenario(scene_file_name='./utamp/scenes/panda_stacking.ttt')
    utamp = driver.UTAMP(scene_file_name='./utamp/scenes/panda_blocks_tiles-5.ttt')

    print("objects presently in scene:", utamp.objects_in_sim)

objects presently in scene: ['blue_tile', 'yellow_tile', 'green_tile', 'red_tile', 'blue_block_1', 'green_block_1', 'yellow_block_1', 'yellow_block_2', 'yellow_block_3', 'red_block_1', 'red_block_2']


In [109]:
# -- use the LLM as a planner to acquire a task plan:

# -- we are going to feed the LLM with the prompt of coming up with a plan using the pre-defined skills:
llm_planner_sys_prompt = open('llm_prompts/llm_planner_system_prompt.txt', 'r').read()
message = [{"role": "system", "content": llm_planner_sys_prompt}]

llm_planner_user_prompt = f"Generate a PDDL task plan for the following task: {user_task}\n"\
    f"The current state of the robot's environment is provided as the following dictionary: {utamp.perform_sensing(method=-1)}."

print(llm_planner_user_prompt)

message.extend([{"role": "user", "content": llm_planner_user_prompt}])

_, response = prompt_LLM(given_prompt=message, model_name=openai_models[-1], verbose=False)
print(response)



  -- Acquiring object properties from CoppeliaSim...: 100%|██████████| 11/11 [00:00<00:00, 240.93it/s]


Generate a PDDL task plan for the following task: I have 3 yellow blocks, 2 red blocks, 1 green block, and 1 blue block. There is a red tile, a yellow tile, and a blue tile on the table. Which blocks can be stacked in their corresponding tiles based on colour?
The current state of the robot's environment is provided as the following dictionary: {'on': {'blue_tile': 'air', 'yellow_tile': 'air', 'green_tile': 'air', 'red_tile': 'air', 'blue_block_1': 'air', 'green_block_1': 'air', 'yellow_block_1': 'air', 'yellow_block_2': 'air', 'yellow_block_3': 'air', 'red_block_1': 'air', 'red_block_2': 'air'}, 'under': {'blue_tile': 'table', 'yellow_tile': 'table', 'green_tile': 'table', 'red_tile': 'table', 'blue_block_1': 'table', 'green_block_1': 'table', 'yellow_block_1': 'table', 'yellow_block_2': 'table', 'yellow_block_3': 'table', 'red_block_1': 'table', 'red_block_2': 'table', 'hand': 'air'}, 'in': {'hand': 'air'}}.
Based on the task and the current state of the robot's environment, the robo